# Get information on long baseline publications

The goal here is to extract the publication information from all long baseline data and send to lance to get citation counts.

In [17]:
from astropy.table import Table
import numpy as np
from astroquery.alma import Alma

In [2]:
mytab = Table.read('ALMA_12m_wL80_and_pubs.csv')

In [3]:
mytab.columns

<TableColumns names=('Project code','Member ous id','ALMA source name','Publications','Array','L80 BL')>

In [9]:
mytab[0:10]

Project code,Member ous id,ALMA source name,Publications,Array,L80 BL
str16,str23,str47,int64,str6,float64
2011.0.00191.S,uid://A001/X62/X2a,Fomalhaut b,2,12m,88.86359
2011.0.00101.S,uid://A001/X74/X29,GRB021004,2,12m,85.9068
2011.0.00131.S,uid://A001/X6f/X10,R Scl,5,12m,89.274315
2011.0.00397.S,uid://A001/X74/X22d,J041754.10-281655.9,3,12m,92.03943
2011.0.00397.S,uid://A001/X74/X22d,J035448.24-330827.2,3,12m,91.1818
2011.0.00397.S,uid://A001/X74/X22f,J063027.81-212058.6,3,12m,86.96931
2011.0.00397.S,uid://A001/X74/X22f,J061200.23-062209.6,3,12m,86.95875
2011.0.00397.S,uid://A001/X74/X22d,J054930.06-373940.1,3,12m,88.99365
2011.0.00397.S,uid://A001/X74/X22f,J070257.20-280842.3,3,12m,89.15855


In [13]:
idx = mytab['L80 BL'] > 1000.0 # in m

In [14]:
np.sum(mytab['Publications'][idx])

7242

In [15]:
len(mytab)

46317

In [16]:
len(mytab[idx])

3627

In [38]:
len(np.unique(mytab[idx]['Member ous id']))

2196

Okay, so the publications here doesn't include the link to the meta-data which is a pain. But I think I can you alma_query to get the publication bibcodes. 

The field is bib_reference.

So how might i do this? loop through each unique MOUS, if > 1.0 km, save MOUS, L80 to new table. then use the new table to query alma archive and get bib_reference

In [18]:
Alma.help()


Most common ALMA query keywords are listed below. These keywords are part of the ALMA ObsCore model, an IVOA standard for metadata representation (3rd column). They were also present in original ALMA Web form and, for backwards compatibility can be accessed with their old names (2nd column).
More elaborate queries on the ObsCore model are possible with `query_sia` or `query_tap` methods
  Description                       Original ALMA keyword               ObsCore keyword                    
-------------------------------------------------------------------------------------------------------

Position
  Source name (astropy Resolver)    source_name_resolver                SkyCoord.from_name                 
  Source name (ALMA)                source_name_alma                    target_name                        
  RA Dec (Sexagesimal)              ra_dec                              s_ra, s_dec                        
  Galactic (Degrees)                galactic                   

In [19]:
Alma.help_tap()

Table to query is "voa.ObsCore".
For example: "select top 1 * from ivoa.ObsCore"
The scheme of the table is as follows.

  Name                 Type            Unit       Description
------------------------------------------------------------------------------------------
  access_format        char(9)                    Content format of the data
  access_url           char(72*)                  URL to download the data
  antenna_arrays       char(660*)                 Blank-separated list of Pad:Antenna pairs, i.e., A109:DV09 J504:DV02 J505:DV05 for antennas DV09, DV02 and DV05 sitting on pads A109, J504, and J505, respectively.
  asdm_uid             char(32*)                  UID of the ASDM containing this Field.
  authors              char(4000*)                Full list of first author and all co-authors
  band_list            char(30*)                  Space delimited list of bands
  bandwidth            double          Hz         Total Bandwidth
  bib_reference        char(40

In [25]:
result = Alma.query_tap(f"SELECT bib_reference FROM ivoa.obscore WHERE member_ous_uid like 'uid://A001/X62/X2a'").to_table()

In [31]:
test = result[0]

In [34]:
test['bib_reference'].split(' ')

['2012ApJ...750L..21B', '2015MNRAS.447.3936M']